Data Science Capstone

this notebook will be mainly used for the capstone project.

In [1]:
pip install bs4

     |████████████████████████████████| 102kB 2.5MB/s ta 0:00:011
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install geopy

     |████████████████████████████████| 102kB 2.7MB/s ta 0:00:011
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/99/45/d1/14954797e2a976083182c2e7da9b4e924509e59b6e5c661061
Successfully built geographiclib
Note: you may need to restart the kernel to use updated packages.


Preperation

In [33]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [4]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


<h1>Then I will download data from wikipedia and transfrom html to Dataframe

In [5]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data=requests.get(url)
soup=BeautifulSoup(data.content)
data_can = pd.DataFrame(columns=['Postal Code','borough','neighbor'])
print(data_can.head())
i=1
for line in soup.find_all('tr'):
    line = line.find_all('td')

    if len(line) == 3 and line[1].string != 'Not assigned':
        Neighborhood=line[2].string
        Bourough=line[1].string
#         print('is{}'.format(line[2].string))
        if line[2].string == 'Not assigned'or line[2].string ==None or line[2].string=='Not assigned\n':
            Neighborhood = Bourough + str(i)
            i += 1
        data_can = data_can.append(pd.Series({'Postal Code':line[0].string,'borough':Bourough,'neighbor':Neighborhood}),ignore_index=True)
#         print('line is %s'%line_info
# print(data_can.iloc[0])
for i in range(211):
    if data_can['borough'].iloc[i]!=data_can['neighbor'].iloc[i]:
        print('............')
        print(data_can.iloc[i])

Empty DataFrame
Columns: [Postal Code, borough, neighbor]
Index: []
............
Postal Code            M3A
borough         North York
neighbor       North York1
Name: 0, dtype: object
............
Postal Code            M4A
borough         North York
neighbor       North York2
Name: 1, dtype: object
............
Postal Code                  M5A
borough         Downtown Toronto
neighbor       Downtown Toronto3
Name: 2, dtype: object
............
Postal Code                  M5A
borough         Downtown Toronto
neighbor       Downtown Toronto4
Name: 3, dtype: object
............
Postal Code            M6A
borough         North York
neighbor       North York5
Name: 4, dtype: object
............
Postal Code            M6A
borough         North York
neighbor       North York6
Name: 5, dtype: object
............
Postal Code              M7A
borough         Queen's Park
neighbor       Queen's Park7
Name: 6, dtype: object
............
Postal Code           M9A
borough         Etobicoke
neighb

In [6]:
data_can.shape

(211, 3)

<h2>I chose to directly download the coordinates data

In [7]:
ll_can=pd.read_csv('Geospatial_Coordinates.csv')

In [8]:
df3 = pd.merge(data_can,ll_can,on='Postal Code',how='left')

In [9]:
df3.head()

,Postal Code,borough,neighbor,Latitude,Longitude
0,M3A,North York,North York1,43.753259,-79.329656
1,M4A,North York,North York2,43.725882,-79.315572
2,M5A,Downtown Toronto,Downtown Toronto3,43.654260,-79.360636
3,M5A,Downtown Toronto,Downtown Toronto4,43.654260,-79.360636
4,M6A,North York,North York5,43.718518,-79.464763


In [10]:
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [11]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df3['Latitude'],
                                           df3['Longitude'],
                                           df3['borough'],
                                           df3['neighbor']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

In [12]:
neighborhoods = df3
northyork_data = neighborhoods[neighborhoods['borough'] == 'North York'].reset_index(drop=True)
northyork_data.shape

(38, 5)

In [13]:
CLIENT_ID = 'ACRKVGW24PGQSDX3UWRFY5Y2BDI4QMUQLBJGASNSFM21ZNDD' # your Foursquare ID
CLIENT_SECRET = 'D3XZKMVQSGQG5OSTO3JZVALMUG44NXBRNJZGCPSDI1BNMFOQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ACRKVGW24PGQSDX3UWRFY5Y2BDI4QMUQLBJGASNSFM21ZNDD
CLIENT_SECRET:D3XZKMVQSGQG5OSTO3JZVALMUG44NXBRNJZGCPSDI1BNMFOQ


In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        LIMIT =10
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
#         if results == []:
#             venues_list.append([
#             name, 
#             lat, 
#             lng, 
#             None, 
#             0, 
#             0,  
#             None])
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
# northyork_data.head()
# LIMIT =10
# radius=500
# lat,lng = 43.789053,-79.408493
#         # create the API request URL
# url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#     CLIENT_ID, 
#     CLIENT_SECRET, 
#     VERSION, 
#     lat, 
#     lng, 
#     radius, 
#     LIMIT)
# results= requests.get(url)
# results.json()

In [16]:
northyork_venues = getNearbyVenues(names=northyork_data['neighbor'],
                                   latitudes=northyork_data['Latitude'],
                                   longitudes=northyork_data['Longitude']
                                  )
northyork_venues['Neighborhood'].unique()

North York1
North York2
North York5
North York6
Don Mills North

Glencairn

North York19
Don Mills South

North York30
North York31
Downsview North

North York32
Fairview

North York36
Oriole

North York37
North York38
North York46
North York47
Downsview East

Silver Hills

North York56
North York57
North York60
North Park

Upwood Park

North York61
North York64
North York65
Downsview Central

North York66
Lawrence Manor East

North York70
North York71
Willowdale South

Downsview Northwest

York Mills West

North York82


array(['North York1', 'North York2', 'North York5', 'North York6',
       'Don Mills North\n', 'Glencairn\n', 'North York19',
       'Don Mills South\n', 'North York30', 'North York31',
       'Downsview North\n', 'North York32', 'Fairview\n', 'North York36',
       'Oriole\n', 'North York37', 'North York38', 'North York46',
       'North York47', 'Downsview East\n', 'Silver Hills\n',
       'North York56', 'North York57', 'North York60', 'North Park\n',
       'Upwood Park\n', 'North York61', 'Downsview Central\n',
       'North York66', 'Lawrence Manor East\n', 'North York70',
       'North York71', 'Willowdale South\n', 'Downsview Northwest\n',
       'York Mills West\n', 'North York82'], dtype=object)

In [17]:
df=northyork_venues.groupby('Neighborhood').count()
print(df.index)
dif=set(northyork_data['neighbor'])-set(df.index)
for i in dif:
    print(northyork_data[northyork_data['neighbor']==i])
    print('-----')

Index(['Don Mills North\n', 'Don Mills South\n', 'Downsview Central\n',
       'Downsview East\n', 'Downsview North\n', 'Downsview Northwest\n',
       'Fairview\n', 'Glencairn\n', 'Lawrence Manor East\n', 'North Park\n',
       'North York1', 'North York19', 'North York2', 'North York30',
       'North York31', 'North York32', 'North York36', 'North York37',
       'North York38', 'North York46', 'North York47', 'North York5',
       'North York56', 'North York57', 'North York6', 'North York60',
       'North York61', 'North York66', 'North York70', 'North York71',
       'North York82', 'Oriole\n', 'Silver Hills\n', 'Upwood Park\n',
       'Willowdale South\n', 'York Mills West\n'],
      dtype='object', name='Neighborhood')
   Postal Code     borough      neighbor   Latitude  Longitude
27         M2M  North York  North York64  43.789053 -79.408493
-----
   Postal Code     borough      neighbor   Latitude  Longitude
28         M2M  North York  North York65  43.789053 -79.408493
-----

In [18]:
print('There are {} uniques categories.'.format(len(northyork_venues['Venue Category'].unique())))

There are 72 uniques categories.


<h2>We should notice that Northyork64 and Northyork65 don't have venues, so we have to manually add full zero to the data in case that kmeans.labels_ don't match the shape of neighborhoods

In [21]:
# one hot encoding
northyork_onehot = pd.get_dummies(northyork_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
northyork_onehot['Neighborhood'] = northyork_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [northyork_onehot.columns[-1]] + list(northyork_onehot.columns[:-1])
northyork_onehot = northyork_onehot[fixed_columns]
print(northyork_onehot.shape)
for i in dif:
    northyork_onehot.loc[northyork_onehot.shape[0]]=[i]+[0 for x in range(northyork_onehot.shape[1]-1)] 

(207, 73)


In [23]:
northyork_grouped = northyork_onehot.groupby('Neighborhood').mean().reset_index()
northyork_grouped.shape

(38, 73)

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = northyork_grouped['Neighborhood']

for ind in np.arange(northyork_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(northyork_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Don Mills North\n,Caribbean Restaurant,Café,Gym / Fitness Center,Japanese Restaurant,Baseball Field,Deli / Bodega,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
1,Don Mills South\n,Clothing Store,Bike Shop,Japanese Restaurant,Italian Restaurant,Gym,Sporting Goods Shop,Discount Store,Restaurant,Concert Hall,Beer Store
2,Downsview Central\n,Food Truck,Baseball Field,Diner,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Deli / Bodega,Discount Store
3,Downsview East\n,Airport,Park,Women's Store,Deli / Bodega,Clothing Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
4,Downsview North\n,Coffee Shop,Diner,Bank,Bridal Shop,Deli / Bodega,Restaurant,Middle Eastern Restaurant,Sushi Restaurant,Fast Food Restaurant,Dog Run


In [26]:
# set number of clusters
kclusters = 5

northyork_grouped_clustering = northyork_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(northyork_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]
len(kmeans.labels_)

38

In [27]:
# add clustering labels
# neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
print(northyork_data.shape)
northyork_merged = northyork_data
northyork_merged = northyork_merged.rename(columns={'neighbor': 'Neighborhood'})
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
northyork_merged = northyork_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
northyork_merged['label'] = kmeans.labels_
northyork_merged.head() # check the last columns!

(38, 5)


,Postal Code,borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,label
0,M3A,North York,North York1,43.753259,-79.329656,Food & Drink Shop,Park,Filipino Restaurant,Fast Food Restaurant,Empanada Restaurant,Electronics Store,Dog Run,Discount Store,Diner,Caribbean Restaurant,0
1,M4A,North York,North York2,43.725882,-79.315572,Coffee Shop,Hockey Arena,Portuguese Restaurant,Intersection,Women's Store,Convenience Store,Clothing Store,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,0
2,M6A,North York,North York5,43.718518,-79.464763,Clothing Store,Women's Store,Miscellaneous Shop,Boutique,Coffee Shop,Vietnamese Restaurant,Furniture / Home Store,Accessories Store,Fast Food Restaurant,Electronics Store,4
3,M6A,North York,North York6,43.718518,-79.464763,Clothing Store,Women's Store,Miscellaneous Shop,Boutique,Coffee Shop,Vietnamese Restaurant,Furniture / Home Store,Accessories Store,Fast Food Restaurant,Electronics Store,1
4,M3B,North York,Don Mills North\n,43.745906,-79.352188,Caribbean Restaurant,Café,Gym / Fitness Center,Japanese Restaurant,Baseball Field,Deli / Bodega,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,0


In [34]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(northyork_merged['Latitude'], northyork_merged['Longitude'], northyork_merged['Neighborhood'], northyork_merged['label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters